In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv('gurgaon_data_after_feature_selection.csv')

In [49]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,40.0,3,2,2.0,1.0,850.0,0,0,0,1.0,1.0,0.82
1,0.0,101.0,2,2,2.0,1.0,1226.0,1,0,0,1.0,2.0,0.95
2,0.0,111.0,2,2,1.0,1.0,1000.0,0,0,0,1.0,0.0,0.32
3,0.0,105.0,3,4,4.0,3.0,1615.0,1,0,1,0.0,2.0,1.60
4,0.0,4.0,2,2,1.0,3.0,582.0,0,1,0,0.0,2.0,0.48


In [50]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [51]:
X = df.drop(columns=['price'])
y = df['price']

In [52]:
from sklearn.model_selection import KFold , cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

In [53]:
from sklearn.linear_model import LinearRegression

In [54]:
df.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category', 'price'],
      dtype='object')

In [55]:
columns_to_encode =  ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [56]:
y_transformed = np.log1p(y)

In [84]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first',handle_unknown= 'ignore'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [85]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['property_type', 'bedRoom', 'bathroom',
                                  'built_up_area', 'servant room',
                                  'store room']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['sector', 'balcony', 'agePossession',
                                  'furnishing_type', 'luxury_category',
                                  'floor_category'])])

In [86]:
pipe = Pipeline(
    [
        ('Preprocessor' , preprocessor),
        ('regressor' , LinearRegression())
    ]
)

In [87]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipe , X , y_transformed , cv = kfold , scoring = 'r2')

C:\Users\vinay\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\vinay\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [71]:
scores.mean()

0.8511064845505883

In [89]:
from sklearn.svm import SVR

In [90]:
pipe = Pipeline(
    [
        ('Preprocessor' , preprocessor),
        ('regressor' , SVR(kernel = 'rbf'))
    ]
)

In [91]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores2 = cross_val_score(pipe , X , y_transformed , cv = kfold , scoring = 'r2')

C:\Users\vinay\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\vinay\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [92]:
scores2

array([0.87503472, 0.89193385, 0.87845937, 0.91426519, 0.89805001,
       0.89055434, 0.88296822, 0.88647802, 0.89285161, 0.86201349])

In [93]:
scores2.mean()

0.8872608821935355

In [94]:
scores.std()

0.021033391777797582

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2)

In [110]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [111]:
y_pred = pipe.predict(X_test)

In [112]:
y_pred = np.expm1(y_pred)

In [113]:
from sklearn.metrics import r2_score , mean_absolute_error

In [114]:
r2_score(np.expm1(y_test) , y_pred)

0.8206652716791467

In [115]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.5203172488366455